# Introduction to Artificial Intelligence
Laboratory 1



Exercise 1 Route searching 
1.	Create a set of cities (as points) with coordinates x, y on a plane with height as z coordinate. The cost of going from city A to city B is equal to the Euclidean distance between two cities, if there exists a road. You should define scenarios according to two criteria: 
a.	There are all the direct connections / c.a. 80% of possible connections
b.	The problem is symmetrical / asymmetrical (in asymmetrical – going up is height +10%, going down: -10%)
You should choose the coordinates randomly from the range <-100, 100> for x,y and <0, 50> for z.
2.	Represent the created map as a weighted (directed) graph, where cities are the nodes and roads are the edges of the graph.
3.	In the created scene, solve the traveling salesman problem: The salesman starts from a chosen city and has to visit every city exactly once before returning to the starting city. The goal is to find a path with the lowest cost.
In the problem, we define state as a partial or full path from the starting city and the corresponding state. You should represent the search problem in a form of state tree.
a.	Implement a full search of the tree, using BFS and DFS methods.
b.	Approximate the solution using greedy search (NN and Dijkstra)
c.	Solve/approximate the solution using A* with inadmissible/admissible heuristics
d.	Approximate the solution using ACO algorithm
4.	Test each algorithm, in each scenario, for n=5…20 cities, in terms of the found path cost, time and memory consumption.


In [2]:
import math

def euclidean_distance(a,b)->float:  
    """ 
    points as a 
    tuple or list for example a = (1,2,3) , b = (4,5,6)
    """
    import math
    dimension = len(a)
    if (dimension !=len(b)):
        print("Dimension missmatch")
        return 0 
    dist = 0
    for i in range(dimension):
        dist += math.pow(a[i]-b[i],2)
    return math.sqrt(dist)
    
    
    
def euclidean_distance_assymetrical(a,b)->float:  
    """ 
    points as a 
    tuple or list for example a = (1,2,3) , b = (4,5,6)
    """
    
    dimension = len(a)
    if (dimension !=len(b)):
        print("Dimension missmatch")
        return 0 
    dist = 0
    for i in range(dimension-1): # let's assume last dimesion as high 
        dist += math.pow(a[i]-b[i],2)
        
    if ((a[-1]-b[-1]) > 0):   # a is higher than b so distance from a to b 
        dist += math.pow(0.9*(a[-1]-b[-1]),2)
    else:
        dist += math.pow(1.1*(a[-1]-b[-1]),2)
    
    return math.sqrt(dist)


In [3]:
import random
class CitiesMap():
    def __init__(self,n,seed = None):
        """
        initialize with number of cities to include in graph 
        """    
        
        if seed is not None:
            random.seed(seed)
        self.seed = seed
        self.n = n 
        self.cities = []
        self.routes = [[0 for _ in range(n)] for _ in range(n)]
        # we can represent routes between cities as a matrix with n x n shape 
        for i in range(n):
            self.cities.append((random.randint(-101,101),random.randint(-101,101),random.randint(0,51)))

        # now let's make sure that we don't have duplicated cities so we'll get derired number of unique cities
        
        len_check = len(set(self.cities))
        
        while (len_check != n):
            self.cities.append((random.randint(-101,101),random.randint(-101,101),random.randint(0,51)))
            len_check = len(set(self.cities))
        self.cities = list(set(self.cities))
    
    
    def calculate_distance(self,symmetrical: bool = True , connections: float = 1):
        """
        Fill matrix with distnaces between city i and j  0 < i,j < n with 0 on main diagonal , conenctions 1 means all cities will be connected
        0.5 only half of them and so on 
        
        """
             
        if (symmetrical):
            if (connections==1):   
                for i in range(self.n):
                    for j in range(i+1 , self.n):  # to not double calcualte
                        
                        distance = euclidean_distance(self.cities[i], self.cities[j])
                    
                        # Store the route only once, as it is undirected
                        self.routes[i][j] = distance
                        self.routes[j][i] = distance
                return self.routes
            else:
                # in this case we nned to delete number of connections that equals to connections * (n^2-n)/2 
                
                n_to_delete = int(round((1 - connections) * ((self.n**2 - self.n) / 2),0))

                indexes_list = []    
                                
                for i in range(self.n):
                    for j in range(i+1 , self.n):  # to not double calcualte
                        
                        indexes_list.append((i,j))
                        
                        distance = euclidean_distance(self.cities[i], self.cities[j])
                    
                        # Store the route only once, as it is undirected
                        self.routes[i][j] = distance
                        self.routes[j][i] = distance
            
                indexes_to_delete = random.sample(indexes_list,n_to_delete)
                
                for i in indexes_to_delete:
                    # in the list that we iterate over we have pair of matrix indexes when let's say i means position with 0 index in matrix rows, j column
                    self.routes[i[0]][i[1]] = 0     # deletes i,i
                    self.routes[i[1]][i[0]] = 0        # deletes j,i 
                    
                    
                
                
                return self.routes
                
                
                
        else: 
            if (connections==1):  
                for i in range(self.n):
                    for j in range(self.n):
                        self.routes[i][j] = euclidean_distance_assymetrical(self.cities[i], self.cities[j])
                        
                return self.routes  
            else:
                # in this case we nned to delete number of connections that equals to connections * (n^2-n) # assymetrical 
                
                n_to_delete = int(round((1 - connections) * ((self.n**2 - self.n) / 2),0))
                indexes_list = []    
                                
                for i in range(self.n):
                    for j in range(self.n):
                        # however with connections to delete we'll keep approach from previous one as I understand that lack connections means no road between two points 
                        # no matter how long 
                        
                        self.routes[i][j] = euclidean_distance_assymetrical(self.cities[i], self.cities[j])
                        
                        # adds to indexes base only ones from above main diagonal 
                        if (j>i):
                            indexes_list.append((i,j))
                    
                    
                        
                indexes_to_delete = random.sample(indexes_list,n_to_delete)
                
                
                for i in indexes_to_delete:
                    # in the list that we iterate over we have pair of matrix indexes 
                    self.routes[i[0]][i[1]] = 0     # deletes i,i
                    self.routes[i[1]][i[0]] = 0
                    
                    
                
                
                return self.routes
                
                
    
    


In [4]:
from queue import PriorityQueue
from sys import maxsize
class RouteSearch:
    def __init__(self,cities_map : CitiesMap):
        self.nodes = cities_map.cities
        self.size = cities_map.n
        self.edges = cities_map.routes
        # self.states = []
        self.max = maxsize
    def dfs_all_paths(self, starting_node: int):
        """
        Perform DFS to explore all possible paths, both complete and incomplete.
        
        Parameters:
        starting_node : int - the node from which to start the search
        
        Returns:
        list : A list of paths. Path can be incomplete or complete.
        """
        # Stack for DFS
        stack = [[starting_node]]
        all_paths = []
    
        while stack:
            path = stack.pop()
            last_node = path[-1]
            
            
            # Record the current (incomplete or complete) path and its cost
            all_paths.append(path)

            # If the path includes all cities, check if it can return to the start
            if len(path) == self.size:
                return_to_start_cost = self.edges[last_node][starting_node]
                if return_to_start_cost != 0:     # to make sure that we can return to start from there
                    # Record the completed cycle (path returns to the start)
                    all_paths.append(path + [starting_node])

            # Explore all unvisited neighbors (continue DFS)
            for neighbor in range(self.size - 1, -1, -1):  # reverse order to simulate stack behavior
                if neighbor not in path and self.edges[last_node][neighbor] != 0:
                    new_path = path + [neighbor]
                    stack.append(new_path)
                    
        return all_paths  

    def bfs_all_paths(self, starting_node: int):
        """
        Perform BS to explore all possible paths, both complete and incomplete.
        
        Parameters:
        starting_node : int - the node from which to start the search
        
        Returns:
        list : A list of paths. Path can be incomplete or complete.
        """
        from collections import deque

        # queue for bfs
        # queue = [[starting_node]]  # with this by hand implementation this algorithm was useless already around n = 10. As I read due to O(n) pop(0) opearation. 
        queue = deque([[starting_node]]) 

        all_paths = []

        while queue:
            # path = queue.pop(0)   # take first element from list (queue)
            path = queue.popleft()
            last_node = path[-1]
            
            # Record the current (incomplete or complete) path and its cost
            all_paths.append(path)

            # If the path includes all cities, check if it can return to the start
            if len(path) == self.size:
                return_to_start_cost = self.edges[last_node][starting_node]   # to make sure that we can return to start from there
                if return_to_start_cost != 0:
                    # Record the completed cycle (path returns to the start)
                    all_paths.append(path + [starting_node])

                continue

            # Explore all unvisited neighbors (continue bfs)
            for neighbor in range(0, self.size):  # normal queue order
                if neighbor not in path and self.edges[last_node][neighbor] != 0:
                    new_path = path + [neighbor]
                    queue.append(new_path)

        return all_paths
    def get_distance_uncomplete_cycle(self,cycle):
        distance = 0
        path_len = len(cycle)
        for index in range(path_len-1):
         # it's in fact len(state_tree) - 1 -> exactly what we need in this approach
            distance += self.edges[cycle[index]][cycle[index+1]]  # we need to extract ith and ith+1 element of our cycle 
        return distance
    
    
    def get_distance(self,cycle):
        distance = 0
        for index in range(self.size):  # it's in fact len(state_tree) - 1 -> exactly what we need in this approach
            distance += self.edges[cycle[index]][cycle[index+1]]  # we need to extract ith and ith+1 element of our cycle 
        return distance
    
    def get_min_distance(self,state_tree):
        """ Requires state tree object from for example dfs or bfs algo, calculate distance only for full paths"""
        
        min_distance = self.max
        shortest_route = []
        for cycle in state_tree:
            if (len(cycle)==self.size + 1):  # we calculate distance only in case of full cycle
                distance = self.get_distance(cycle)
            
                if (distance < min_distance):
                    min_distance = distance
                    shortest_route = cycle

        return (shortest_route,min_distance)
    
    def nn(self,start):
        
        path = [start]
        
        while(len(path)!=self.size):
            dist = maxsize  # initialize distance as maxsize
            node = path[-1]  # get last node from path as a new 'starting node'
            target = -1  # initialize target as a number from outside matrix index
            for neighbour in range(self.size):   # iterate over neighbours of 'new' target                 
                cur_dist = self.edges[node][neighbour] 
                if ((neighbour not in path) & (0 < cur_dist < dist)):    # if path exists and is shorter than previous lower from this starting point 
                    target = neighbour   # store neighbour
                    dist = cur_dist   # store shortest distance 
                    
            if target == -1:    # we got in the dead end 
           
                return path , "Didn't find complete route"
            path.append(target)
        
        if (self.edges[target][start]!=0):    # if we reached there and we can go back to start we found complete circle
            path.append(start)
           
            return path , self.get_distance(path)
        
        # póki co nie testuję nn
      
        return path, "Didn't find complete route - no connection between last node and starting node" 
    
    def heuristic(self,path,heuristic_type):
        
        if (self.size==len(path)):
            return 0 
        
        else:
            if heuristic_type ==  'admissible':
                # minimum for each unvisited node 
                
                unvisited = [node for node in range(self.size) if node not in path]
                min_edges = [] # table for minimum for each unvisited node
                current = path[-1]
                for node in unvisited:
                    node_edges = [self.edges[node][j] for j in range(self.size)
                                  if self.edges[node][j]!=0]
                    # local minimum for edge 
                    min_edges.append(min(node_edges))
                return sum(min_edges) + min(min_edges) # sdding minimum because we need to go back to start so we can safely add that 
            
            elif heuristic_type ==  'inadmissible':
                 # minimum to the neighbour and than straight line to the start
                current = path[-1]
                minimum = maxsize
                for neighbour in range(self.size):
                    if neighbour not in path and self.edges[current][neighbour]!=0:
                        if self.edges[current][neighbour] < minimum:
                            minimum = self.edges[current][neighbour]
                            
                if minimum == maxsize:
                    return 0 
                return minimum * (self.size- len(path) + 1)
                
                
            else:
                raise ValueError("Invalid heuristic type, use 'admissible' or 'inadmissible' eventually None if you want to perform Djikstra instead of A_star")
                
        

    
    
    
    def a_star(self,start,heuristic_type):
        
        pq = PriorityQueue() 
        
        pq.put((0, [start])) # pair, distance path 

        distance , path = pq.get()
        current_node = path[-1]

        while len(path) != self.size:   # because we want to find closed tour in this loop 
            # if self.edges[current_node][start] != 0:
            #     total_distance = distance + self.edges[current_node][start]
            #     complete_path = path + [start]
                
                   
            
            for neighbour in range(self.size):
                if neighbour not in path and self.edges[current_node][neighbour] != 0:
                    distance = self.get_distance_uncomplete_cycle(path)                       
                    new_distance = distance + self.edges[current_node][neighbour]
                    
                    # Create new path
                    new_path = path + [neighbour]
                    
                    if heuristic_type:
                        h = self.heuristic(new_path,heuristic_type)
                        new_distance += h
                    
                    pq.put((new_distance,new_path))
            distance , path = pq.get()
            current_node = path[-1]
            # I will add possibility of going back to start there. It will increase time complexity of loop by some constant but will guarantee that we find complete solution in loop
            # if len(path)==self.size:
            #     # try to close 
            #     if self.edges[current_node][start]!=0:
            #         distance += self.edges[current_node][start]
            #         path = path + [start]
            # else: 
            #     distnace,path = pq.get()  # go back one step and continue loop if we couldn't close loop 
            # #useless because it will go back to same solutions anyway 
            
        # return path, distance 
        if self.edges[current_node][start]!=0:
            return path + [start] , distance + self.edges[current_node][start]
        else:
            print("Can't complete tour")
            return None, None
            
    
    def dijkstra(self,start):
        return self.a_star(start,heuristic_type=None)

    def aco(self, alfa =0.9 , beta = 1.5 , decay = .5, Q = 5,steps = 10, ants = None ,best_half_update= True,random_seed = 42,print_info = True):
        
        if random_seed:
            random.seed(random_seed)
        
        # default to number of cities in the problem
        if ants:
            pass
        else:  
            ants = self.size    
        
        # initialize matrix for pheromones storage 
        phero_table = [[1 for _ in range(self.size)] for _ in range(self.size)]
        shortest_distance = maxsize
        def ant_path(start):
            path = [start]
            path_distance = 0
            
            # we will first create list of possibles continuations and their probs 
            while len(path) < self.size: 
                current = path[-1]

                probs = []
                possible_steps = []
                for neighbour in range(self.size):
                    dist = self.edges[current][neighbour]
                    if ((dist != 0)& (neighbour not in path)):
                        possible_steps.append(neighbour)  # we append possible step together with pheromone of it 
                        prob_temp = phero_table[current][neighbour]**alfa / dist **beta # we store distance there 
                        probs.append(prob_temp)

                if len(possible_steps) == 0: 
                    # print("First if fired")
                    return path,maxsize   # we went to dead end 
                next_step = random.choices(possible_steps,weights=probs)[0]    # we are choosing next step, appending distance and direction 
                path_distance += self.edges[current][next_step]
                path.append(next_step)
                current = next_step
            # try to return to start 
            if (self.edges[current][path[0]]!=0):
                path.append(start)
                path_distance += self.edges[current][path[0]] # back to the start 
            else:
                return path, maxsize  # we will divide by very huge number so we will update pheromon by very low value  
            # if we get there we found complet cycle. Let's return path and  distnace of cycle
            return path, path_distance
            
        
        for step in range(steps):
            # we will place ants in order and if ants>self.size then we will start from the 0 index city again 
            results_table = []
            
            for ant in range(ants):
                results_table.append(ant_path(ant % self.size))

            # also let's keep best solution in step for now just for printing it out
            results_table.sort(key = lambda x: x[1])

            # if update only based on top half of solution 
            if best_half_update:
                results_table = results_table[:len(results_table)//2]
            
            
            # now we need to update phero_table according to formula, first of all we can apply decay rate 
            
            for i in range(self.size):
                for j in range(self.size):
                    phero_table[i][j] = phero_table[i][j] * (1 - decay)
        
            # now update based on result table - solutions generated by ants in previous step 
            for route, distance in results_table: 
                for k in range (len(route)-1):
                    r = route[k]
                    s = route[k+1]   # we extract pairs of city on path of this ant
                    # and update pehero_table in cosent with formula
                    phero_table[r][s] += Q / distance   
            if print_info:                      
                print(f"Step: {step} best route distance: {results_table[0][1]}")
        
            # keep best soluton 
            if results_table[0][1] < shortest_distance:
                shortest_distance = results_table[0][1]
                best_solution = results_table[0][0] , results_table[0][1]
        
        
        
        return best_solution

In [5]:
# test = CitiesMap(20,seed=42)  # this will generate route without connection with nn
test = CitiesMap(10,seed=40)
test.calculate_distance(symmetrical=False,connections=1);

In [6]:
algos = RouteSearch(test)

In [7]:

paths = algos.dfs_all_paths(0)


In [8]:
paths1 = algos.bfs_all_paths(0)

In [9]:
print("Paths generated by DFS:")
paths

Paths generated by DFS:


[[0],
 [0, 1],
 [0, 1, 2],
 [0, 1, 2, 3],
 [0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4, 5],
 [0, 1, 2, 3, 4, 5, 6],
 [0, 1, 2, 3, 4, 5, 6, 7],
 [0, 1, 2, 3, 4, 5, 6, 7, 8],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0],
 [0, 1, 2, 3, 4, 5, 6, 7, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 9, 8],
 [0, 1, 2, 3, 4, 5, 6, 7, 9, 8, 0],
 [0, 1, 2, 3, 4, 5, 6, 8],
 [0, 1, 2, 3, 4, 5, 6, 8, 7],
 [0, 1, 2, 3, 4, 5, 6, 8, 7, 9],
 [0, 1, 2, 3, 4, 5, 6, 8, 7, 9, 0],
 [0, 1, 2, 3, 4, 5, 6, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 8, 9, 7],
 [0, 1, 2, 3, 4, 5, 6, 8, 9, 7, 0],
 [0, 1, 2, 3, 4, 5, 6, 9],
 [0, 1, 2, 3, 4, 5, 6, 9, 7],
 [0, 1, 2, 3, 4, 5, 6, 9, 7, 8],
 [0, 1, 2, 3, 4, 5, 6, 9, 7, 8, 0],
 [0, 1, 2, 3, 4, 5, 6, 9, 8],
 [0, 1, 2, 3, 4, 5, 6, 9, 8, 7],
 [0, 1, 2, 3, 4, 5, 6, 9, 8, 7, 0],
 [0, 1, 2, 3, 4, 5, 7],
 [0, 1, 2, 3, 4, 5, 7, 6],
 [0, 1, 2, 3, 4, 5, 7, 6, 8],
 [0, 1, 2, 3, 4, 5, 7, 6, 8, 9],
 [0, 1, 2, 3, 4, 5, 7, 6, 8, 9, 0],
 [0, 1, 2, 3, 4, 5, 7, 6, 9],
 [0, 1, 2, 3, 4, 5, 7, 6, 9, 8],
 [0, 1,

In [10]:
print("Paths generated by BFS:")
paths1

Paths generated by BFS:


[[0],
 [0, 1],
 [0, 2],
 [0, 3],
 [0, 4],
 [0, 5],
 [0, 6],
 [0, 7],
 [0, 8],
 [0, 9],
 [0, 1, 2],
 [0, 1, 3],
 [0, 1, 4],
 [0, 1, 5],
 [0, 1, 6],
 [0, 1, 7],
 [0, 1, 8],
 [0, 1, 9],
 [0, 2, 1],
 [0, 2, 3],
 [0, 2, 4],
 [0, 2, 5],
 [0, 2, 6],
 [0, 2, 7],
 [0, 2, 8],
 [0, 2, 9],
 [0, 3, 1],
 [0, 3, 2],
 [0, 3, 4],
 [0, 3, 5],
 [0, 3, 6],
 [0, 3, 7],
 [0, 3, 8],
 [0, 3, 9],
 [0, 4, 1],
 [0, 4, 2],
 [0, 4, 3],
 [0, 4, 5],
 [0, 4, 6],
 [0, 4, 7],
 [0, 4, 8],
 [0, 4, 9],
 [0, 5, 1],
 [0, 5, 2],
 [0, 5, 3],
 [0, 5, 4],
 [0, 5, 6],
 [0, 5, 7],
 [0, 5, 8],
 [0, 5, 9],
 [0, 6, 1],
 [0, 6, 2],
 [0, 6, 3],
 [0, 6, 4],
 [0, 6, 5],
 [0, 6, 7],
 [0, 6, 8],
 [0, 6, 9],
 [0, 7, 1],
 [0, 7, 2],
 [0, 7, 3],
 [0, 7, 4],
 [0, 7, 5],
 [0, 7, 6],
 [0, 7, 8],
 [0, 7, 9],
 [0, 8, 1],
 [0, 8, 2],
 [0, 8, 3],
 [0, 8, 4],
 [0, 8, 5],
 [0, 8, 6],
 [0, 8, 7],
 [0, 8, 9],
 [0, 9, 1],
 [0, 9, 2],
 [0, 9, 3],
 [0, 9, 4],
 [0, 9, 5],
 [0, 9, 6],
 [0, 9, 7],
 [0, 9, 8],
 [0, 1, 2, 3],
 [0, 1, 2, 4],
 [0, 1, 2, 5],
 [0,

In [11]:
algos.get_min_distance(paths)   # based on route from dfs or bfs 

([0, 3, 5, 2, 1, 9, 8, 6, 4, 7, 0], 735.3157640026147)

In [12]:
algos.nn(0)

([0, 3, 5, 1, 2, 9, 6, 8, 4, 7, 0], 766.845408778468)

In [13]:
algos.dijkstra(0)

([0, 3, 5, 2, 1, 9, 8, 6, 4, 7, 0], 735.3157640026147)

In [14]:
algos.a_star(0,heuristic_type='admissible')

([0, 3, 5, 2, 1, 9, 8, 6, 4, 7, 0], 735.3157640026147)

In [15]:
algos.a_star(0,heuristic_type='inadmissible')

([0, 3, 5, 2, 1, 9, 8, 6, 4, 7, 0], 735.3157640026147)

In [16]:
algos.aco(steps=10,decay=0.9,Q=10,best_half_update=True,random_seed=42,print_info=False)

([9, 1, 2, 5, 3, 0, 7, 4, 6, 8, 9], 736.437276608299)

In [19]:
import pandas as pd 
import time 
def compare_algorithms(n , symmetrical , conenctions,seed, brute_force):
    """Not recommended to set brute force True for n > 12/13 """
    city_map = CitiesMap(n=n,seed=seed)
    city_map.calculate_distance(symmetrical=symmetrical,connections=conenctions)
    algos_space = RouteSearch(city_map)
    algo_name = []
    path = []
    shortest_path = []
    timer = []
    if brute_force:
        # brute forceprint("")
         print("Brute force")
         start = time.time()
         
         paths_ = algos_space.dfs_all_paths(0)
         dfs_distance = algos_space.get_min_distance(paths_)   # based on route from dfs or bfs 
         end = time.time()
         length = end - start
         algo_name.append("Brute Force")
         path.append(dfs_distance[0])
         shortest_path.append(dfs_distance[1])
         timer.append(length)
         
         # nn 
        

         
         print("NN")
         start = time.time()
         nn_distance = algos_space.nn(0)
         end = time.time()
         length = end - start
         algo_name.append("Nearest Neighbour")
         path.append(nn_distance[0])
         shortest_path.append(nn_distance[1])
         timer.append(length)
         
         # djikstra will be calculated for n<= 14 ~ 6 minutes
         if n <= 15:
            print("Djikstra")
            start = time.time()
            djisktra_distance = algos_space.dijkstra(0)
            end = time.time()
            length = end - start
            length = end - start
            algo_name.append("Djikstra")
            path.append(djisktra_distance[0])
            shortest_path.append(djisktra_distance[1])
            timer.append(length)
         
         # a _start admmissible heuristic 
         print("A*adm")
         start = time.time()
         a_star_adm = algos_space.a_star(0,heuristic_type="admissible")
         end = time.time()
         length = end - start
         algo_name.append("A* admissible heuristic")
         path.append(a_star_adm[0])
         shortest_path.append(a_star_adm[1])
         timer.append(length)
          # a _start inadmmissible heuristic 
          
         if n<=18:
            print("A*inadm")
            start = time.time()
            a_star_inadm = algos_space.a_star(0,heuristic_type="inadmissible")
            end = time.time()
            length = end - start
            algo_name.append("A* inadmissible heuristic")
            path.append(a_star_inadm[0])
            shortest_path.append(a_star_inadm[1])
            timer.append(length)
            
         # aco 
         print("Aco")
         start = time.time()
         aco_distance = algos_space.aco(steps=200,decay=0.9,Q=10,best_half_update=True,random_seed=42,print_info=False)
         end = time.time()
         length = end - start
         algo_name.append("ACO")
         path.append(aco_distance[0])
         shortest_path.append(aco_distance[1])
         timer.append(length)
         
    else:
        # nn 
        
         
         print("NN")
         start = time.time()
         nn_distance = algos_space.nn(0)
         end = time.time()
         length = end - start
         algo_name.append("Nearest Neighbour")
         path.append(nn_distance[0])
         shortest_path.append(nn_distance[1])
         timer.append(length)
         
        # Djikstra
         if n <= 15:        
            print("Djikstra")        
            start = time.time()
            
            djisktra_distance = algos_space.dijkstra(0)
            end = time.time()
            length = end - start
            algo_name.append("Djikstra")
            path.append(djisktra_distance[0])
            shortest_path.append(djisktra_distance[1])
            timer.append(length)
         
         # a _start admmissible heuristic 
         print("A*adm")
         start = time.time()
         a_star_adm = algos_space.a_star(0,heuristic_type="admissible")
         end = time.time()
         length = end - start
         algo_name.append("A* admissible heuristic")
         path.append(a_star_adm[0])
         shortest_path.append(a_star_adm[1])
         timer.append(length)
          # a _start inadmmissible heuristic 
         if n<=18:
            print("A*inadm")
            start = time.time()
            a_star_inadm = algos_space.a_star(0,heuristic_type="inadmissible")
            end = time.time()
            length = end - start
            algo_name.append("A* inadmissible heuristic")
            path.append(a_star_inadm[0])
            shortest_path.append(a_star_inadm[1])
            timer.append(length)
         
         # aco 
         print("aco")        
         start = time.time()
         aco_distance = algos_space.aco(steps=10,decay=0.9,Q=10,best_half_update=True,random_seed=42,print_info=False)
         end = time.time()
         length = end - start
         algo_name.append("ACO")
         path.append(aco_distance[0])
         shortest_path.append(aco_distance[1])
         timer.append(length)
    df = pd.DataFrame(data = {"Name of algo":algo_name,"Path of solution":path,"Distance of solution":shortest_path,"Time of calculations":timer})
    return df 

In [19]:
df = compare_algorithms(n = 12 , symmetrical= True , conenctions= 1 , brute_force= True, seed = 84);


In [20]:
df

,Name of algo,Path of solution,Distance of solution,Time of calculations
0,Brute Force,"[0, 8, 10, 6, 3, 11, 9, 2, 7, 1, 5, 4, 0]",638.920322,116.327139
1,Nearest Neighbour,"[0, 4, 5, 8, 10, 11, 6, 3, 9, 7, 1, 2, 0]",707.852457,0.000000
2,Djikstra,"[0, 4, 5, 8, 10, 6, 3, 11, 9, 2, 7, 1, 0]",662.279908,1.380604
3,A* admissible heuristic,"[0, 4, 5, 8, 10, 6, 3, 11, 9, 7, 2, 1, 0]",700.430019,0.205996
4,A* inadmissible heuristic,"[0, 5, 4, 8, 10, 6, 3, 11, 9, 2, 7, 1, 0]",675.240424,0.105000
5,ACO,"[4, 5, 8, 10, 11, 6, 3, 9, 2, 7, 1, 0, 4]",677.290646,0.052000


In [21]:
df1 = compare_algorithms(n = 12 , symmetrical= False , conenctions= 1 , brute_force= True, seed = 84);
df1

,Name of algo,Path of solution,Distance of solution,Time of calculations
0,Brute Force,"[0, 4, 5, 1, 7, 2, 9, 11, 3, 6, 10, 8, 0]",639.082634,115.003957
1,Nearest Neighbour,"[0, 4, 5, 8, 10, 11, 6, 3, 9, 7, 1, 2, 0]",708.000686,0.000000
2,Djikstra,"[0, 4, 5, 8, 10, 6, 3, 11, 9, 2, 7, 1, 0]",660.658700,1.280529
3,A* admissible heuristic,"[0, 4, 5, 8, 10, 6, 3, 11, 9, 7, 1, 2, 0]",692.266768,0.181000
4,A* inadmissible heuristic,"[0, 5, 4, 8, 10, 6, 3, 11, 9, 2, 7, 1, 0]",675.195100,0.101001
5,ACO,"[11, 3, 6, 10, 9, 2, 7, 1, 0, 4, 5, 8, 11]",703.899731,0.052998


In [27]:
df2 = compare_algorithms(n = 12 , symmetrical= True , conenctions= 0.8 , brute_force= True, seed = 12);
df2 

,Name of algo,Path of solution,Distance of solution,Time of calculations
0,Brute Force,"[0, 11, 8, 2, 7, 3, 1, 10, 6, 4, 5, 9, 0]",691.607929,13.344259
1,Nearest Neighbour,"[0, 11, 8, 3, 2, 7, 1, 10, 6, 4, 5, 9, 0]",711.280430,0.000000
2,Djikstra,"[0, 11, 8, 2, 7, 3, 1, 10, 6, 4, 5, 9, 0]",691.607929,3.795606
3,A* admissible heuristic,"[0, 5, 4, 11, 8, 2, 7, 3, 1, 10, 6, 9, 0]",729.616883,0.102000
4,A* inadmissible heuristic,"[0, 2, 7, 3, 10, 6, 1, 8, 11, 4, 5, 9, 0]",743.512242,0.006999
5,ACO,"[4, 1, 10, 6, 2, 3, 7, 8, 11, 0, 9, 5, 4]",715.045247,0.051003


In [28]:
df3 = compare_algorithms(n = 12 , symmetrical= False , conenctions= 0.8 , brute_force= True, seed = 12);
df3 

,Name of algo,Path of solution,Distance of solution,Time of calculations
0,Brute Force,"[0, 11, 8, 2, 7, 3, 1, 10, 6, 4, 5, 9, 0]",690.328294,11.616668
1,Nearest Neighbour,"[0, 11, 8, 3, 2, 7, 1, 10, 6, 4, 5, 9, 0]",713.368555,0.000000
2,Djikstra,"[0, 11, 8, 2, 7, 3, 1, 10, 6, 4, 5, 9, 0]",690.328294,1.243444
3,A* admissible heuristic,"[0, 5, 4, 11, 8, 2, 7, 3, 1, 10, 6, 9, 0]",727.789085,0.099001
4,A* inadmissible heuristic,"[0, 2, 7, 3, 10, 6, 1, 8, 11, 4, 5, 9, 0]",743.691665,0.008000
5,ACO,"[5, 9, 0, 11, 8, 7, 3, 2, 6, 10, 1, 4, 5]",715.932688,0.049000


In [25]:
df44 = compare_algorithms(n = 12 , symmetrical= True , conenctions= 0.8 , brute_force= True, seed = 144);
df44

Brute force
NN
Djikstra
A*adm
A*inadm
Aco


,Name of algo,Path of solution,Distance of solution,Time of calculations
0,Brute Force,"[0, 5, 6, 10, 2, 4, 7, 3, 8, 11, 1, 9, 0]",629.171337,26.009381
1,Nearest Neighbour,"[0, 5, 6, 10, 4, 2, 3, 8, 1, 11, 7, 9, 0]",686.923263,0.000000
2,Djikstra,"[0, 5, 6, 10, 2, 4, 7, 3, 8, 11, 1, 9, 0]",629.171337,1.604184
3,A* admissible heuristic,"[0, 5, 6, 10, 4, 2, 11, 1, 8, 3, 7, 9, 0]",651.958374,0.034026
4,A* inadmissible heuristic,"[0, 9, 1, 11, 2, 8, 3, 7, 4, 10, 6, 5, 0]",642.485945,0.097076
5,ACO,"[2, 4, 7, 3, 8, 11, 1, 9, 0, 5, 6, 10, 2]",629.171337,0.052605


In [26]:
df44 = compare_algorithms(n = 12 , symmetrical= True , conenctions= 0.8 , brute_force= True, seed = 13);
df44

Brute force
NN
Djikstra
A*adm
A*inadm
Aco


,Name of algo,Path of solution,Distance of solution,Time of calculations
0,Brute Force,"[0, 2, 11, 4, 6, 1, 7, 3, 8, 10, 9, 5, 0]",797.428012,7.646248
1,Nearest Neighbour,"[0, 5, 9, 10, 1, 6, 3, 7, 11, 2, 8]",Didn't find complete route,0.000000
2,Djikstra,"[0, 5, 9, 10, 8, 3, 7, 1, 6, 2, 11, 4, 0]",839.809331,2.192080
3,A* admissible heuristic,"[0, 5, 9, 10, 8, 3, 7, 1, 6, 4, 11, 2, 0]",797.428012,0.316868
4,A* inadmissible heuristic,"[0, 5, 9, 10, 8, 3, 7, 1, 6, 2, 11, 4, 0]",839.809331,0.149623
5,ACO,"[6, 4, 11, 2, 0, 5, 10, 9, 7, 3, 8, 1, 6]",814.544341,0.051543


In [29]:
df15 = compare_algorithms(n = 15 , symmetrical= True , conenctions= 1 , brute_force= False, seed = 12);
df15

,Name of algo,Path of solution,Distance of solution,Time of calculations
0,Nearest Neighbour,"[0, 10, 14, 1, 6, 2, 3, 9, 13, 7, 12, 4, 11, 5...",774.419818,0.000000
1,Djikstra,"[0, 8, 5, 11, 4, 14, 10, 3, 9, 2, 6, 1, 13, 7,...",831.791105,690.129657
2,A* admissible heuristic,"[0, 8, 5, 11, 4, 14, 10, 3, 9, 2, 6, 1, 13, 12...",832.815187,19.459567
3,A* inadmissible heuristic,"[0, 10, 3, 9, 2, 6, 1, 13, 14, 8, 5, 11, 4, 7,...",920.065210,11.936055
4,ACO,"[12, 7, 13, 1, 6, 2, 9, 3, 10, 14, 0, 8, 5, 11...",750.399602,0.005000


In [27]:
df16 = compare_algorithms(n = 16 , symmetrical= True , conenctions= 1 , brute_force= False  , seed = 42)

NN
A*adm
A*inadm
aco


In [9]:
df16

,Name of algo,Path of solution,Distance of solution,Time of calculations
0,Nearest Neighbour,"[0, 7, 9, 13, 15, 14, 3, 12, 10, 5, 6, 2, 8, 1...",1081.908062,0.000000
1,A* admissible heuristic,"[0, 13, 7, 9, 15, 14, 6, 3, 12, 10, 5, 1, 2, 1...",922.844819,8.057138
2,A* inadmissible heuristic,"[0, 13, 7, 4, 11, 1, 10, 5, 6, 3, 12, 2, 8, 14...",947.021476,78.828534
3,ACO,"[9, 15, 14, 6, 5, 12, 3, 10, 1, 2, 8, 11, 4, 1...",909.530165,0.006508


In [28]:
df161 = compare_algorithms(n = 16 , symmetrical= True , conenctions= 0.8 , brute_force= False  , seed = 42)
df161

NN
A*adm
A*inadm
aco


,Name of algo,Path of solution,Distance of solution,Time of calculations
0,Nearest Neighbour,"[0, 7, 9, 13, 15, 14, 3, 12, 10, 5, 6, 2, 8, 1...",1081.908062,0.000000
1,A* admissible heuristic,"[0, 7, 13, 9, 15, 14, 6, 5, 10, 12, 3, 2, 8, 1...",1020.110307,3.357245
2,A* inadmissible heuristic,"[0, 7, 13, 4, 11, 8, 2, 3, 12, 10, 1, 5, 6, 14...",925.041697,14.501328
3,ACO,"[3, 12, 10, 1, 5, 6, 14, 15, 0, 7, 9, 13, 4, 1...",944.498429,0.005507


In [29]:
df181 = compare_algorithms(n = 18 , symmetrical= True , conenctions= 0.8 , brute_force= False  , seed = 42)
df181

NN
A*adm
A*inadm
aco


,Name of algo,Path of solution,Distance of solution,Time of calculations
0,Nearest Neighbour,"[0, 5, 11, 9, 13, 3, 12, 14, 7, 4, 6, 2, 17, 1...",1142.032216,0.001000
1,A* admissible heuristic,"[0, 5, 11, 9, 13, 3, 4, 6, 2, 12, 14, 7, 16, 1...",1081.483908,41.979874
2,A* inadmissible heuristic,"[0, 13, 3, 12, 14, 7, 4, 6, 2, 5, 11, 1, 15, 8...",1078.435228,12.105454
3,ACO,"[5, 0, 13, 3, 6, 4, 7, 12, 14, 2, 9, 16, 17, 1...",1065.395958,0.008512


In [20]:
df20 = compare_algorithms(n = 20 , symmetrical= True , conenctions= 1 , brute_force= False  , seed = 42)
df20

NN
A*adm
aco


,Name of algo,Path of solution,Distance of solution,Time of calculations
0,Nearest Neighbour,"[0, 12, 10, 4, 8, 9, 6, 11, 14, 1, 7, 3, 5, 15...",1117.296975,0.000000
1,A* admissible heuristic,"[0, 12, 10, 4, 8, 2, 6, 11, 14, 1, 7, 3, 5, 16...",921.443636,679.384122
2,ACO,"[11, 6, 4, 10, 12, 0, 9, 8, 2, 14, 1, 7, 3, 15...",971.213378,0.011505
